James Gardner, March 2022

In [ ]:
# overkill but whatever
from constants import *
from basic_benchmarking import *

### Testing whether ET_ET1 automatically adds the other two (it doesn't)

In [ ]:
# comparing ET_ET1 versus all three
net1 = network.Network(['ET_ET1',])
net2 = network.Network(['ET_ET1','ET_ET2','ET_ET3'])

f = np.geomspace(1e0, 1e4, int(1e4))
wf_model_name, wf_other_var_dic = 'tf2_tidal', None
# injection parameters, for GW170817
inj_params = {
    'Mc':    1.188, 
    'eta':   0.2485, 
    'chi1z': 0,
    'chi2z': 0,
    'DL':    40, 
    'tc':    0, 
    'phic':  0,
    'iota':  PI/4,
    'ra':    PI/4, 
    'dec':   PI/4,
    'psi':   PI/4, 
    'gmst0': 0, 
    'lam_t': 800, 
    'delta_lam_t': 0
}
deriv_symbs_string = 'Mc eta chi1z chi2z DL tc phic iota ra dec psi lam_t'
conv_log = ('Mc', 'DL', 'lam_t')
conv_cos = ('iota', 'dec')
use_rot = 1
net_only = 1

generate_symbolic_derivatives(wf_model_name, wf_other_var_dic, deriv_symbs_string, ['ET1', 'ET2', 'ET3'], use_rot)

for net in (net1, net2):
    net.set_wf_vars(wf_model_name=wf_model_name)
    net.set_net_vars(
        f=f, inj_params=inj_params,
        deriv_symbs_string=deriv_symbs_string,
        conv_cos=conv_cos, conv_log=conv_log,
        use_rot=use_rot
    )
    basic_network_benchmarking(net, numerical_over_symbolic_derivs=False)

In [ ]:
print(f"Comparing net1: {net_label_styler(net1.label)} to net2: {net_label_styler(net2.label)}\n(Ratios given as net2/net1)\nRatio of integrated SNR: {net2.snr/net1.snr:.4g}, ratio squared: {(net2.snr/net1.snr)**2:.4g}\nRatio of 90%-credible sky area: {net2.errs['sky_area_90']/net1.errs['sky_area_90']:.4g}")

### Plotting sensitivity curves

In [ ]:
# initialise network
# fixed location for comparing PSDs (noise doesn't depend on it but detector response does, except eq3.6 suggests otherwise, see confusion below)
loc = 'H'
# detector technologies
tecs_2GET = ['aLIGO',
             'A+', 'V+', 'K+',
             'Voyager-CBO', 'Voyager-PMO',
             'ET']
tecs_CE = [f'CE{n}-{Larm}-{opt}' for n in [1, 2] for opt in ['CBO', 'PMO'] for Larm in range(10, 50, 10)]

# omni-network
network_spec = [f'{tec}_{loc}' for tec in tecs_2GET + tecs_CE] # using list concatenation (+) 
locs = [loc for _ in network_spec]
# comparing ET_H to ET_ET1 --> in either case, in network.detectors only one ET appears and the sensitivity is the same
network_spec.append('ET_ET1')
net = network.Network(network_spec)

# frequency range
# f = np.geomspace(1, 4e3, int(1e4))
f = np.arange(5, 20, 1/16)

# choose the desired waveform 
# wf_model_name, wf_other_var_dic = 'tf2', None
# net.set_wf_vars(wf_model_name=wf_model_name)

# injection parameters, for GW150914
inj_params = {
    'Mc':    30.9,
    'eta':   0.247,
    'chi1z': 0,
    'chi2z': 0,
    'DL':    475,
    'tc':    0,
    'phic':  0,
    'iota':  PI/4,
    'ra':    PI/4,
    'dec':   PI/4,
    'psi':   PI/4,
    'gmst0': 0,
}

# derivative settings
deriv_symbs_string = 'Mc eta DL tc phic iota ra dec psi'
conv_cos = ('iota', 'dec')
conv_log = ('Mc', 'DL')
# network settings
use_rot = 1
only_net = 1

net.set_net_vars(
    f=f, inj_params=inj_params,
    deriv_symbs_string=deriv_symbs_string,
    conv_cos=conv_cos, conv_log=conv_log,
    use_rot=use_rot
    )
print('Network initialised')

In [ ]:
# just psd required
net.setup_ant_pat_lpf_psds()

In [ ]:
# plot sensitivity curves together, a la Fig 5 in Borhanian 2021
colours_2GET = [*[(1, 0, 1, i) for i in np.arange(0.25, 1.25, 0.25)], (0, 1, 0, 0.5), (0, 1, 0, 1), (1, 0.5, 0, 1), (0, 0, 0, 1)]
colours_CE = [(*rgb, Larm/40) for rgb in ((1, 0, 0), (0, 0, 1)) for Larm in range(10, 50, 10)]

plt.rcParams.update({'font.size': 16})
fig, axs = plt.subplots(2, 3, figsize=(16, 8), gridspec_kw={'wspace':0.05, 'hspace':0.1})
for i, detector in enumerate(net.detectors):
    # detector signal response is sky-position and polarisation dependent (in honours I assumed perfect alignment)
#     axs[1].loglog(detector.f, np.abs(detector.hf))
    # confused, formula for PSD looks like noise spectrum with no trace of signal transfer function
    # SNR is calculated as H/Sn, which suggests that Sn is the spectrum of noise
    # however, the units and scale are correct for the ASD of NSR in the plot
    if detector.tec in tecs_2GET:
        tec_type = 0
        color=colours_2GET[min(i, len(colours_2GET) - 1)]
    # else, is in CE
    else:
        tec_type = int(detector.tec[2])
        color=colours_CE[tecs_CE.index(detector.tec) % 8]
                
    if i == len(net.detectors) - 1:
        linestyle = '--'
    else:
        linestyle = '-'        
    axs[0,tec_type].loglog(detector.f, np.sqrt(detector.psd), label=detector.tec, color=color, linestyle=linestyle)
    axs[1,tec_type].loglog(detector.f, np.sqrt(detector.psd), label=detector.tec, color=color, linestyle=linestyle)

# top row
for i, ax in enumerate(axs[0]):
    ax.legend(fontsize=14, ncol=2, columnspacing=1, handlelength=1)
    ax.set(xlim=(f[0], f[-1]), ylim=(1e-25, 1e-15))
    ax.grid(which='both', axis='both', color='lightgrey')
    ax.set_xticklabels([])
    if i > 0:
        ax.set_yticklabels([]) 
    
# bottom row
for i, ax in enumerate(axs[1]):
    # frequency range truncated to range specified in PSD
    ax.set(xlabel='frequency, f / Hz', xlim=(f[0], f[-1]), ylim=(2e-25, 1e-23))
    ax.grid(which='both', axis='both', color='lightgrey')    
    if i > 0:
        ax.set_yticklabels([])
    
axs[0,0].set_ylabel(r'sensitivity / $\mathrm{Hz}^{-1/2}$')
axs[1,0].set_ylabel(r'sensitivity / $\mathrm{Hz}^{-1/2}$')

plt.show()
fig.savefig('plots/sensitivity_curves.pdf')
plt.close(fig)

In [ ]:
# plotting to compare ET curve to Design Report Update 2020

plt.rcParams.update({'font.size': 16})

fig, ax = plt.subplots()
detector = net.detectors[-1]
ax.loglog(detector.f, np.sqrt(detector.psd), 'k--')

ax.set(xlabel='frequency, f / Hz', ylabel=r'sensitivity / $\mathrm{Hz}^{-1/2}$', xlim=(1e0, 1e4), ylim=(1e-25, 1e-21))
ax.grid(which='both', axis='both', color='lightgrey')    

plt.show()
fig.savefig('plots/ET_ET1_sensitivity_curve.pdf')
plt.close(fig)

In [ ]:
# checking behaviour from 5 to 1024 Hz to see if the frequency array is being truncated (it is and the minimum frequency for V+ and aLIGO, the limiting pair, is 10 Hz)
print('Detector key, min freq, max freq, num freqs\n')
for i, key in enumerate(net.det_keys):
    det = net.detectors[i]
    print(f'{key}, {det.f[0]:.2g}, {det.f[-1]:.4g}, {len(det.f)}')

In [ ]:
# to-do: resolve this confusion about noise, signal, and sensitivity